In [1]:
import epidag as dag
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
scr = '''
PCore test {
    prob ~ beta(1, 1)
    x ~ binom(n, prob)    
}
'''

bn = dag.bayes_net_from_script(scr)

In [3]:
from epidag.fitting import Fitter, BayesianModel


In [4]:
class BinBeta(BayesianModel):
    def __init__(self, bn, data):
        BayesianModel.__init__(self, bn, pars=['prob'])
        self.Data = data
        
    @property
    def has_exact_likelihood(self):
        return True

    def evaluate_distance(self, pars):
        return - self.evaluate_likelihood(pars)

    def evaluate_likelihood(self, pars):
        li = 0
        pars = dict(pars)
        for d in self.Data:
            pars.update(d)
            li += self.BN['x'].evaluate(pars)
        return li



In [5]:
data = [
    {'id': 1, 'n': 10, 'x': 4},
    {'id': 2, 'n': 20, 'x': 7}
]

model = BinBeta(bn, data)


In [99]:
d = bn['prob'].get_distribution()

In [103]:
p = sample(bn)
bn['x'].get_distribution()

KeyError: 'n'

In [6]:
p = model.sample_prior()
model.evaluate_distance(p), model.evaluate_likelihood(p)

(4.864246461955711, -4.864246461955711)

In [7]:
p = {'prob': 11/30}
model.evaluate_distance(p), model.evaluate_likelihood(p)

(3.1093340676380965, -3.1093340676380965)

In [37]:
alg = dag.fitting.ABC(p_test = 0.05)

res = alg.fit(model, n_post = 200)

INFO:ABC:Testing threshold
INFO:ABC:Detected epsilon = -3.144
INFO:ABC:Collecting posterior parameters
INFO:ABC:Completed


In [14]:
import numpy as np

In [46]:
def find_eps(ds, eps):
    e0 = (ds < eps0).mean()
    et = 0.9 * e0
    ds = list(ds)
    ds.sort(key = lambda x : -x)
    ds = np.array(ds)
    for eps1 in ds:
        e1 = (ds < eps1).mean()
        if e1 <= et:
            return eps1
    else:
        return eps

In [84]:
n_post = 100
p_thres = 0.8
ess_thres = n_post * p_thres
post = list()
while len(post) < n_post:
    p = model.sample_prior()
    di = model.evaluate_likelihood(p)
    if np.isfinite(di):
        p.LogLikelihood = - di
        post.append(p)

d0 = np.array([-p.LogLikelihood for p in post])

wts = np.ones(n_post) / n_post

eps0 = float('inf')
n_round = n_stay = 0
n_eval = n_post

n_round += 1
eps1 = find_eps(d0, eps0)


if eps1 > eps0:
    n_stay += 1
    eps1 = eps0
else:
    n_stay = 0

act_np0 = d0 < eps0
act_np1 = d0 < eps1
a = act_np0 > 0

wts[a] *= act_np1[a] / act_np0[a]
wts[1- act_np0] = 0
wts /= wts.sum()

if ess_thres * sum(wts * wts) > 1:
    assert sum(wts > 0) > 2
    alive = wts > 0
    ind = [k for k, v in enumerate(alive) if v]
    re_ind = rd.choice(ind, size=n_post, replace=T, p=wts[wts > 0])
    
    post = [post[i].clone() for i in re_ind]
    d1 = d0[re_ind]
    wts = np.ones(n_post) / n_post
else:
    d1 = d0

In [87]:
def wt_sd(vs, wts):
    vs = np.array(vs)
    wts = wts / wts.sum()
    mu = (wts * vs) / len(wts)
    
    return np.sqrt(np.sum(wts * (vs - mu) * (vs - mu)))



In [93]:
act_np0 = d0 < eps1
alive = act_np0 > 0

## MH step

tau = {p: wt_sd([d[p] for d in post], wts) for p in model.ParameterNodes}

tau

{'prob': 0.6307853783349535}

In [134]:
np.isinf(float('-inf'))

True

In [130]:
ss = Steppers(model)

In [131]:
p = model.sample_prior()
print(p, p.LogPrior)
p = ss.mutate(model, p, {'prob': 0.2})
print(p, p.LogPrior)


prob: 0.58265 None
prob: 0.636021 0.0


In [79]:
alive = wts > 0
ind = [k for k, v in enumerate(alive) if v]
re_ind = rd.choice(ind, 10, p=wts[wts > 0])
post = [post[i].clone() for i in re_ind]

In [34]:
if ess_tres * np.sum(wts * wts):
    if sum(wts > 0) < 2:
        # error
        return 
    alive = wts > 0
    ind = [i for i, al in enumerate(alive) if al]
    re_index = 
    
if (ess_thres * sum(wts ^2) > 1) {
  stopifnot(sum(wts > 0) > 2)
  alive <- wts > 0
  ind <- (1:n_posterior)[alive]
  re_index <- sample(ind, n_posterior, prob = wts[alive], replace = T)
  theta_1 <- theta_0[re_index]

  posteriors <- posteriors[re_index]

  d_1 <- d_0[re_index]
  wts <- rep(1, n_posterior) / n_posterior
} else {
  theta_1 <- theta_0
  d_1 <- d_0
}

In [38]:
fit = dag.fitting.MCMC(dm)
fit.fit(1000)
fit.summarise_posterior()

AttributeError: module 'epidag.fitting' has no attribute 'MCMC'

In [15]:
fit = dag.fitting.ABC(dm)
fit.fit(100, test_p=0.1)
fit.summarise_posterior()

INFO:root:Initialising
INFO:root:Testing threshold
INFO:root:Fitting
INFO:root:Gathering posteriori
INFO:root:Finished
          al     be        prob
count  100.0  100.0  100.000000
mean     1.0    1.0    0.366832
std      0.0    0.0    0.034689
min      1.0    1.0    0.311369
25%      1.0    1.0    0.334778
50%      1.0    1.0    0.367846
75%      1.0    1.0    0.397980
max      1.0    1.0    0.424331
N       100.000000
ESS      99.575397
Dbar      6.373474
pD        0.004413
DIC       6.377887
dtype: float64


In [9]:
scr = '''
PCore ex {
    sun ~ unif(0, 1)
    b0 = 1
    heat ~ exp(sun)
    x1 ~ norm(0, 1)
    x2 ~ norm(0, 1)
    b1 ~ unif(10, 20)
    mu1 = b0 + b1*x1 + b2*x2
    mu2 = b0 + b1*x1
    y1 ~ norm(mu1, sd)
    y2 ~ norm(mu2, 1)
}

'''

bn = dag.bayes_net_from_script(scr)
hie = {
    'g': ['sun', 'agent'],
    'agent': ['x1', 'x2','mu1', 'mu2']
}
sm = dag.as_simulation_core(bn, hie=hie, random=['x2'])


h = dag.form_hierarchy(bn, hie)

h.print()

g(heat, b2, sd, b1, b0, sun)
-- agent(mu1, y2, x1, mu2, y1, x2)


In [10]:
pc = sm.generate(exo={'sd'})
ag1 = pc.breed('a1', 'agent')

In [19]:
ag1.get_sampler('y1')(b2= 4, sd =3)

-1.5826879957652007

In [13]:
pc.get_sampler('heat')()

0.7308904995335411

In [12]:
ng = dag.form_hierarchy(bn, hie)
dag.analyse_node_type(bn, ng, True)


Group g
Exogenous    : ['b2', 'sd']
Can be fixed: ['b1', 'b0', 'sun', 'heat']
Can be random: ['sun', 'heat']
Can be actors: ['heat']
--Group agent
  Exogenous    : []
  Can be fixed: ['x1', 'mu2', 'x2', 'y2']
  Can be random: ['x1', 'mu2', 'x2', 'y2', 'y1', 'mu1']
  Can be actors: ['y2', 'y1']


{'g': (['b2', 'sd'], ['b1', 'b0'], ['sun'], ['heat'], [], []),
 'agent': ([], [], ['x1', 'mu2', 'x2'], ['y2'], ['y1'], ['mu1'])}